In [10]:
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph

Note: you may need to restart the kernel to use updated packages.


In [16]:
%pip install -qU "langchain[google-genai]" --use-deprecated=legacy-resolver

Note: you may need to restart the kernel to use updated packages.


In [12]:
%pip install -qU langchain-google-genai

Note: you may need to restart the kernel to use updated packages.


In [17]:
%pip install -qU langchain-chroma --use-deprecated=legacy-resolver

Note: you may need to restart the kernel to use updated packages.


In [14]:
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [18]:
#force downgrade urllib due to errors with modernized version
%pip install urllib3==1.26.18

Note: you may need to restart the kernel to use updated packages.


In [19]:
import getpass
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["LANGSMITH_TRACING"] = "true"
if not os.environ.get("LANGSMITH_API_KEY"):
    os.environ["LANGSMITH_API_KEY"] = getpass.getpass("Enter Langsmith API Key")

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma

llm = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

vector_store = Chroma(
    collection_name="example_collection",
    embedding_function=embeddings,
    persist_directory="./chroma_langchain_db",  # Where to save data locally, remove if not necessary
)

print("YAY")

YAY


In [ ]:
#Load papers 
file_path = "dnutg/paper1.pdf"